# The goal of this notebook is to detect objects through the Intel RealSense camera, and output a stream showing boxes (and distances) of the detected objects.

In [8]:
# Import Statements
import pyrealsense2 as rs
import numpy as np
import cv2
import math

RealSense Configuration Code:

In [9]:
# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30) # depth stream
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30) # color stream
align = rs.align(rs.stream.color) # align both streams to same pov

In [10]:
# Enable visualizer and filters for later use
colorizer = rs.colorizer()
spatial = rs.spatial_filter()
spatial.set_option(rs.option.filter_magnitude, 5)
spatial.set_option(rs.option.filter_smooth_alpha, 1)
spatial.set_option(rs.option.filter_smooth_delta, 50)
spatial.set_option(rs.option.holes_fill, 3)
hole_filling = rs.hole_filling_filter()
depth_to_disparity = rs.disparity_transform(True)
disparity_to_depth = rs.disparity_transform(False)

OpenCV Detection Configration Code:

In [11]:
# Image detection size
expected = 300
inScaleFactor = 0.007843
meanVal = 127.53

In [12]:
net = cv2.dnn.readNetFromTensorflow('frozen_inference_graph.pb', 'graph.pbtxt') # pretrained net

swapRB = True
classNames = { 0: 'background',
    1: 'person', 2: 'bicycle', 3: 'car', 4: 'motorcycle', 5: 'airplane', 6: 'bus',
    7: 'train', 8: 'truck', 9: 'boat', 10: 'traffic light', 11: 'fire hydrant',
    13: 'stop sign', 14: 'parking meter', 15: 'bench', 16: 'bird', 17: 'cat',
    18: 'dog', 19: 'horse', 20: 'sheep', 21: 'cow', 22: 'elephant', 23: 'bear',
    24: 'zebra', 25: 'giraffe', 27: 'backpack', 28: 'umbrella', 31: 'handbag',
    32: 'tie', 33: 'suitcase', 34: 'frisbee', 35: 'skis', 36: 'snowboard',
    37: 'sports ball', 38: 'kite', 39: 'baseball bat', 40: 'baseball glove',
    41: 'skateboard', 42: 'surfboard', 43: 'tennis racket', 44: 'bottle',
    46: 'wine glass', 47: 'cup', 48: 'fork', 49: 'knife', 50: 'spoon',
    51: 'bowl', 52: 'banana', 53: 'apple', 54: 'sandwich', 55: 'orange',
    56: 'broccoli', 57: 'carrot', 58: 'hot dog', 59: 'pizza', 60: 'donut',
    61: 'cake', 62: 'chair', 63: 'couch', 64: 'potted plant', 65: 'bed',
    67: 'dining table', 70: 'toilet', 72: 'tv', 73: 'laptop', 74: 'mouse',
    75: 'remote', 76: 'keyboard', 77: 'cell phone', 78: 'microwave', 79: 'oven',
    80: 'toaster', 81: 'sink', 82: 'refrigerator', 84: 'book', 85: 'clock',
    86: 'vase', 87: 'scissors', 88: 'teddy bear', 89: 'hair drier', 90: 'toothbrush' }  

In [13]:
def get_output_layers(net):
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    return output_layers

Final Stream Output:

In [16]:
# Start streaming
profile = pipeline.start(config)

try:
    while True:

        # Wait for a coherent pair of frames: depth and color
        frames = pipeline.wait_for_frames()
        frames = align.process(frames)
        
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()
        
        if not depth_frame or not color_frame:
            continue

        # filter depth stream: depth2disparity -> spatial -> disparity2depth -> hole_filling
        depth_frame = depth_to_disparity.process(depth_frame)
        depth_frame = spatial.process(depth_frame)
        depth_frame = disparity_to_depth.process(depth_frame)
        depth_frame = hole_filling.process(depth_frame)

        # Convert images to numpy arrays
        depth_image = np.asanyarray(colorizer.colorize(depth_frame).get_data())
        color_image = np.asanyarray(color_frame.get_data())
        
        # crop color image for detection
        height, width = color_image.shape[:2]
        expected = 300
        aspect = width / height
        resized_color_image = cv2.resize(color_image, (round(expected * aspect), expected))
        crop_start = round(expected * (aspect - 1) / 2)
        crop_color_img = resized_color_image[0:expected, crop_start:crop_start+expected]

        # Perform object detection through net
        blob = cv2.dnn.blobFromImage(crop_color_img, inScaleFactor, (expected, expected), meanVal, False)
        net.setInput(blob)
        detections = net.forward("detection_out")

        label = detections[0,0,0,1]
        conf  = detections[0,0,0,2]
        xmin  = detections[0,0,0,3]
        ymin  = detections[0,0,0,4]
        xmax  = detections[0,0,0,5]
        ymax  = detections[0,0,0,6]
        
        if (conf < .5):
            # Stack both images horizontally
            images = np.hstack((color_image, depth_image))

            # Show images
            cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
            cv2.imshow('RealSense', images)
            key = cv2.waitKey(1)

            # Press esc or 'q' to close the image window
            if key & 0xFF == ord('q') or key == 27:
                cv2.destroyAllWindows()
                break
            
            continue
        
        className = classNames[int(label)]

        # Calculate box coordinates of detected object
        scale = height / expected
        xmin_depth = int((xmin * expected + crop_start) * scale)
        ymin_depth = int((ymin * expected) * scale)
        xmax_depth = int((xmax * expected + crop_start) * scale)
        ymax_depth = int((ymax * expected) * scale)
        xmin_depth,ymin_depth,xmax_depth,ymax_depth
        
        # Calculate depth of object
        depth = np.asanyarray(depth_frame.get_data())
        # Crop depth data:
        depth = depth[math.floor((xmax_depth+xmin_depth)/2-1):math.ceil((xmax_depth+xmin_depth)/2+1),math.floor((ymax_depth+ymin_depth)/2-1):math.ceil((ymax_depth+ymin_depth)/2+1)].astype(float)

        # Get data scale from the device and convert to meters
        depth_scale = profile.get_device().first_depth_sensor().get_depth_scale()
        depth = depth * depth_scale
        dist,_,_,_ = cv2.mean(depth)        
        
        # Draw square on depth and color streams
        cv2.rectangle(depth_image, (xmin_depth, ymin_depth), 
            (xmax_depth, ymax_depth), (255, 255, 255), 2)
        cv2.rectangle(color_image, (xmin_depth, ymin_depth), 
            (xmax_depth, ymax_depth), (255, 255, 255), 2)
        cv2.putText(color_image, className+" @ "+"{:.2f}".format(dist)+"meters away", 
            (xmin_depth, ymin_depth),
            cv2.FONT_HERSHEY_COMPLEX, 0.5, (255,255,255))
        
        # Stack both images horizontally
        images = np.hstack((color_image, depth_image))

        # Show images
        cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
        cv2.imshow('RealSense', images)
        key = cv2.waitKey(1)
        
        # Press esc or 'q' to close the image window
        if key & 0xFF == ord('q') or key == 27:
            cv2.destroyAllWindows()
            break
        
finally:

    # Stop streaming
    pipeline.stop()